In [1]:
using LinearAlgebra
using Plots

In [2]:
fi = ComplexF64(1.0im)

0.0 + 1.0im

In [3]:
function vec2hermite(v::Vector{Float64})
    N = round(Int, sqrt(length(v)))
    H = zeros(ComplexF64, N, N)
    for i in 1:N
        for j in i:N
            l = N*(i-1) + 2j - i^2
            if(i == j)
                H[i,j] = v[l]
            else
                H[i,j] = v[l-1] + fi*v[l]
            end 
        end
    end
    H = Hermitian(H)
    return H
end

vec2hermite (generic function with 1 method)

In [4]:
function vec2unitary(v::Vector{Float64}, τ::Float64)
    H = vec2hermite(v)
    U = exp(fi*(τ*H))
    return U
end

vec2unitary (generic function with 1 method)

In [5]:
function make_unitary(N::Int, τ::Float64)
    v = rand(Float64, N^2)
    U = vec2unitary(v, τ)
    return U
end

make_unitary (generic function with 1 method)

In [6]:
function norm!(m::Hermitian{ComplexF64, Matrix{ComplexF64}})
    T = real(tr(m))
    m = m./T
end

norm! (generic function with 1 method)

In [7]:
function make_rand_dm(dim::Int)
    ρ_vec = rand(Float64, dim^2)
    rt_ρ = vec2hermite(ρ_vec)
    ρ = Hermitian(norm!(Hermitian(rt_ρ*rt_ρ')))
    return ρ
end

make_rand_dm (generic function with 1 method)

In [8]:
test_U = make_unitary(4, 1.0e0)
II = test_U * test_U'
println([real(II[i,i]) for i in 1:4])

[0.9999999999999999, 1.0000000000000007, 1.0000000000000002, 1.0]


In [9]:
function ehot(vs::Vector{ComplexF64}, i::Int, e_dim::Int)
    s_dim = length(vs)
    ve = zeros(ComplexF64, e_dim*s_dim)
    ve[(s_dim*(i-1)+1):(s_dim*i)] = vs
    #v2 = vcat(vs, ve)
    return ve
end

ehot (generic function with 1 method)

In [10]:
function make_ev(s_ev::Matrix{ComplexF64}, e_dim::Int)
    #e_vec::Vector{Matrix{Float64}} = []
    s_vec::Vector{Matrix{ComplexF64}} = []
    s_dim = size(s_ev)[1]
    tot_dim = s_dim * e_dim
    println(s_dim)
    for i in 1:s_dim
        sm = zeros(ComplexF64, tot_dim, e_dim)
        for j in 1:e_dim
            sm[:,j] = ehot(s_ev[:,i], j, e_dim)
            #push!(e_vec, twohot(i, j, s_dim))
            #push!(s_vec, ehot(s_ev[:,i], j, e_dim))
        end
        push!(s_vec, sm)
    end
    #=
    for j in 1:e_dim
        em = zeros(Float64, tot_dim, s_dim)
        for i in 1:s_dim
            em[:,i] = twohot(i, j, s_dim, e_dim)
        end
        push!(e_vec, em)
    end=#
    return s_vec
end

make_ev (generic function with 1 method)

In [11]:
function make_Mk(U::Matrix{ComplexF64}, s_vec::Vector{Matrix{ComplexF64}})
    L = size(U)[1]
    e_dim = length(s_vec)
    s_dim = div(L,e_dim)
    Ms::Vector{Matrix{ComplexF64}} = []
    for j in 1:e_dim
        for k in 1:s_dim
            push!(Ms, (s_vec[k]'*U*s_vec[j]))
        end
    end
    return Ms
end

make_Mk (generic function with 1 method)

In [25]:
#=
function make_Mk(U::Matrix{ComplexF64}, s_vec::Vector{Matrix{ComplexF64}}, e_vec::Vector{Matrix{Float64}})
    L = size(U)[1]
    e_dim = length(s_vec)
    s_dim = div(L,e_dim)
    Ms::Vector{Matrix{ComplexF64}} = []
    for j in 1:e_dim
        for k in 1:s_dim
            push!(Ms, (e_vec[k]'*U*s_vec[j]))
        end
    end
    return Ms
end=#

make_Mk (generic function with 1 method)

In [12]:
struct Dime
    s_dim::Int
    e_dim::Int
    tot_dim::Int
    M_size::Int
end

In [13]:
dim = Dime(2, 2, 4, 16)

Dime(2, 2, 4, 16)

In [14]:
function Λρ(ρ::Hermitian{ComplexF64, Matrix{ComplexF64}}, Ms::Vector{Matrix{ComplexF64}}, es::Vector{Float64}, ds::Dime)
    #ρ_vec = zeros(Float64, tot_dim^2)
    Lρ = zeros(ComplexF64, ds.s_dim, ds.s_dim)
    for i in 1:tot_dim
        Lρ += es[div(i-1,ds.e_dim)+1]*Ms[i]'*ρ*Ms[i]
    end
    return Hermitian(Lρ)
end

Λρ (generic function with 1 method)

In [15]:
testm = make_rand_dm(2)

2×2 Hermitian{ComplexF64, Matrix{ComplexF64}}:
 0.353482+0.0im        0.407204+0.0737964im
 0.407204-0.0737964im  0.646518+0.0im

In [16]:
function KL_divergence(ρ::Hermitian{ComplexF64, Matrix{ComplexF64}}, σ::Hermitian{ComplexF64, Matrix{ComplexF64}})
    return real(tr(ρ*(log(ρ)-log(σ))))
end

KL_divergence (generic function with 1 method)

In [17]:
struct DMs
    s_dm::Hermitian{ComplexF64, Matrix{ComplexF64}}
    e_dm::Hermitian{ComplexF64, Matrix{ComplexF64}}
    s_evs::Matrix{ComplexF64}
    s_es::Vector{Float64}

    U::Matrix{ComplexF64}
    Ms::Vector{Matrix{ComplexF64}}
end

function init_dms(ds::Dime, τ::Float64)
    s_dm = make_rand_dm(ds.s_dim)
    e_dm = make_rand_dm(ds.e_dim)
    s_es, s_evs = eigen(s_dm)
    U = make_unitary(ds.tot_dim, τ)
    s_evsa = make_ev(s_evs, ds.e_dim)
    Ms = make_Mk(U, s_evsa)
    return DMs(s_dm, e_dm, s_evs, s_es, U, Ms)
end

init_dms (generic function with 1 method)

In [18]:
function Λρ(ρ::Hermitian{ComplexF64, Matrix{ComplexF64}}, dms::DMs, ds::Dime)
    #ρ_vec = zeros(Float64, tot_dim^2)
    Lρ = zeros(ComplexF64, ds.s_dim, ds.s_dim)
    for i in 1:ds.tot_dim
        Lρ += dms.s_es[div(i-1,ds.e_dim)+1]*dms.Ms[i]*ρ*dms.Ms[i]'
    end
    return Hermitian(Lρ)
end

Λρ (generic function with 2 methods)

In [19]:
function Λρd(ρ::Hermitian{ComplexF64, Matrix{ComplexF64}}, dms::DMs, ds::Dime)
    #ρ_vec = zeros(Float64, tot_dim^2)
    Lρ = zeros(ComplexF64, ds.s_dim, ds.s_dim)
    for i in 1:ds.tot_dim
        Lρ += dms.s_es[div(i-1,ds.e_dim)+1]*dms.Ms[i]'*ρ*dms.Ms[i]
    end
    return Hermitian(Lρ)
end

Λρd (generic function with 1 method)

In [64]:
function Λρ2(ρ::Hermitian{ComplexF64, Matrix{ComplexF64}}, dms::DMs, ds::Dime)
    #ρ_vec = zeros(Float64, tot_dim^2)
    Lρ = zeros(ComplexF64, ds.s_dim, ds.s_dim)
    for i in 1:tot_dim
        Lρ += dms.s_es[((i-1)%ds.s_dim)+1]*dms.Ms[i]*ρ*dms.Ms[i]'
    end
    return Hermitian(Lρ)
end

Λρ2 (generic function with 1 method)

In [20]:
test_dms = init_dms(dim, 1.0e0)

2


DMs(ComplexF64[0.8075575000711211 + 0.0im 0.33299042775435295 + 0.1573913566811746im; 0.33299042775435295 - 0.1573913566811746im 0.1924424999288789 + 0.0im], ComplexF64[0.6049161521148058 + 0.0im 0.07356778639138147 + 0.36084318482188604im; 0.07356778639138147 - 0.36084318482188604im 0.3950838478851942 + 0.0im], ComplexF64[0.3830642493381821 + 0.18105926439383688im 0.81893268460482 + 0.3870769713404297im; -0.905803137362715 + 0.0im 0.4236988038027277 + 0.0im], [0.02016015173397144, 0.9798398482660287], ComplexF64[0.38574887136313946 - 0.19834037505238605im -0.2734152926163762 - 0.5048318534459927im -0.5250836545471671 - 0.1710747731807561im 0.05591500169368854 - 0.4173030932907965im; -0.13450871301356657 + 0.44798384104689565im 0.42536654188224016 + 0.03062562347662018im -0.6815997639452196 + 0.3255898507189614im -0.162215020356189 + 0.04942224790094818im; -0.1916135067409676 + 0.3440799270334196im 0.05870083341212699 - 0.1657817699235573im 0.35006043070164544 - 0.019321630203218754im 

In [21]:
kl = KL_divergence(test_dms.s_dm, testm)

0.695155266378352

In [22]:
@show lp = Λρ(test_dms.s_dm, test_dms, dim)
@show tr(lp)

lp = Λρ(test_dms.s_dm, test_dms, dim) = ComplexF64[0.46743130830177515 + 0.0im 0.34857140059671166 + 0.18520049088243068im; 0.34857140059671166 - 0.18520049088243068im 0.5325686916982244 + 0.0im]
tr(lp) = 0.9999999999999996


0.9999999999999996

In [28]:
@show lpd = Λρd(test_dms.s_dm, test_dms, dim)
@show tr(lpd)

lpd = Λρd(test_dms.s_dm, test_dms, dim) = ComplexF64[0.4550148877477402 + 0.0im 0.3005315394049507 + 0.20356430613365323im; 0.3005315394049507 - 0.20356430613365323im 0.7208846292815809 + 0.0im]
tr(lpd) = 1.1758995170293212


1.1758995170293212

In [29]:
function nroot_m(ρ::Hermitian{ComplexF64, Matrix{ComplexF64}}, n::Int)
    #ρ_vec = zeros(Float64, tot_dim^2)
    e, v = eigen(ρ)
    en = e.^(1.0e0/n)
    ρ_n = v*Diagonal(en)*v'
    return Hermitian(ρ_n)
end

nroot_m (generic function with 1 method)

In [30]:
function UρUd(U, ρ::Hermitian{ComplexF64, Matrix{ComplexF64}})
    return Hermitian(U*ρ*U')
end

UρUd (generic function with 1 method)

In [35]:
@show test1 = UρUd(nroot_m(test_dms.s_dm, 2), Hermitian(Matrix{ComplexF64}(I, 2, 2)))
test1 - test_dms.s_dm

test1 = UρUd(nroot_m(test_dms.s_dm, 2), Hermitian(Matrix{ComplexF64}(I, 2, 2))) = ComplexF64[0.8075575000711207 + 0.0im 0.3329904277543528 + 0.15739135668117452im; 0.3329904277543528 - 0.15739135668117452im 0.19244249992887885 + 0.0im]


2×2 Hermitian{ComplexF64, Matrix{ComplexF64}}:
 -4.44089e-16+0.0im          -1.66533e-16-8.32667e-17im
 -1.66533e-16+8.32667e-17im  -5.55112e-17+0.0im

In [41]:
function PetzRecovery(ρ::Hermitian{ComplexF64, Matrix{ComplexF64}}, dms::DMs, ds::Dime)
    lρ = Λρ(ρ, dms, ds)
    lρ_n = nroot_m(lρ, -2)
    lρ1 = UρUd(lρ_n, lρ)
    #println(lρ1)
    lρ2 = Λρd(lρ1, dms, ds)
    #println(lρ2)
    ρ_n = nroot_m(ρ, 2)
    lρ3 = UρUd(ρ_n, lρ2)
    return lρ3
end

PetzRecovery (generic function with 1 method)

In [40]:
using BenchmarkTools

In [43]:
@benchmark test = PetzRecovery(test_dms.s_dm, test_dms, dim)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   8.042 μs …   2.648 ms  ┊ GC (min … max):  0.00% … 98.96%
 Time  (median):     12.709 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   17.488 μs ± 105.916 μs  ┊ GC (mean ± σ):  26.77% ±  4.41%

   ▄              ██▇▇▄                                         
  ▄█▅▂▂▂▁▁▁▁▂▂▂▄▅▆██████▅▃▂▂▂▂▄▅▅▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  8.04 μs         Histogram: frequency by time         23.2 μs <

 Memory estimate: 89.36 KiB, allocs estimate: 80.

In [33]:
test_dms.s_dm

2×2 Hermitian{ComplexF64, Matrix{ComplexF64}}:
 0.807558+0.0im        0.33299+0.157391im
  0.33299-0.157391im  0.192442+0.0im